In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
DIR = '/content/drive/My Drive/AG_News/'

Data Preprocessing - 
Converting Letters in vectors 

In [0]:
import numpy as np
import sys
import csv
from torch.utils.data import Dataset
';//'
class MyDataset(Dataset):
    def __init__(self, data_path, max_length):
        self.data_path = data_path
        self.vocabulary = list("abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’’’/\|_@#$%ˆ&*˜‘+-=<>()[]{}")
        self.identity_mat = np.identity(len(self.vocabulary))
        texts, labels = [], []
        with open(data_path) as csv_file:
            reader = csv.reader(csv_file, quotechar='"')
            for idx, line in enumerate(reader):
                text = ""
                for tx in line[1:]:
                    tx = tx.lower()
                    text += tx
                label = int(line[0]) - 1
                texts.append(text)
                labels.append(label)
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.length = len(self.labels)
        self.num_classes = len(set(self.labels))

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        raw_text = self.texts[index]
        data = np.array([self.identity_mat[self.vocabulary.index(i)] for i in list(raw_text) if i in self.vocabulary],
                        dtype=np.float32)
        if len(data) > self.max_length:
            data = data[:self.max_length]
        elif 0 < len(data) < self.max_length:
            data = np.concatenate(
                (data, np.zeros((self.max_length - len(data), len(self.vocabulary)), dtype=np.float32)))
        elif len(data) == 0:
            data = np.zeros((self.max_length, len(self.vocabulary)), dtype=np.float32)
        label = self.labels[index]
        return data, label

Forward Pass of the Network

In [0]:
import torch.nn as nn

class CharacterLevelCNN(nn.Module):
    def __init__(self, n_classes=14, input_length=1014, input_dim=68,
                 n_conv_filters=256,
                 n_fc_neurons=1024):
        super(CharacterLevelCNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv1d(input_dim, n_conv_filters, kernel_size=7, padding=0), nn.ReLU(),
                                   nn.MaxPool1d(3))
        self.conv2 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=7, padding=0), nn.ReLU(),
                                   nn.MaxPool1d(3))
        self.conv3 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU())
        self.conv5 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU())
        self.conv6 = nn.Sequential(nn.Conv1d(n_conv_filters, n_conv_filters, kernel_size=3, padding=0), nn.ReLU(),
                                   nn.MaxPool1d(3))

        dimension = int((input_length - 96) / 27 * n_conv_filters)
        self.fc1 = nn.Sequential(nn.Linear(dimension, n_fc_neurons), nn.Dropout(0.5))
        self.fc2 = nn.Sequential(nn.Linear(n_fc_neurons, n_fc_neurons), nn.Dropout(0.5))
        self.fc3 = nn.Linear(n_fc_neurons, n_classes)

        if n_conv_filters == 256 and n_fc_neurons == 1024:
            self._create_weights(mean=0.0, std=0.05)
        elif n_conv_filters == 1024 and n_fc_neurons == 2048:
            self._create_weights(mean=0.0, std=0.02)

    def _create_weights(self, mean=0.0, std=0.05):
        for module in self.modules():
            if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
                module.weight.data.normal_(mean, std)
                

    def forward(self, input):
        input = input.transpose(1, 2)
        output = self.conv1(input)
        output = self.conv2(output)
        output = self.conv3(output)
        output = self.conv4(output)
        output = self.conv5(output)
        output = self.conv6(output)

        output = output.view(output.size(0), -1)
        output = self.fc1(output)
        output = self.fc2(output)
        output = self.fc3(output)

        return output

In [0]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix

def get_evaluation(y_true, y_prob, list_metrics):
    y_pred = np.argmax(y_prob, -1)
    output = {}
    if 'accuracy' in list_metrics:
        output['accuracy'] = metrics.accuracy_score(y_true, y_pred)
    if 'loss' in list_metrics:
        try:
            output['loss'] = metrics.log_loss(y_true, y_prob)
        except ValueError:
            output['loss'] = -1
    if 'confusion_matrix' in list_metrics:
        #output['confusion_matrix']  = str(confusion_matrix(y_true, y_pred(dim=0)))
        output['confusion_matrix'] = str(metrics.confusion_matrix(y_true, y_pred))
    return output

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import csv

DataSet = DIR + "train.csv"

Batch Size = 128

Max Length = 1014

**manual_seed(seed) → Generator**

Sets the seed for generating random numbers. Returns a torch.Generator object. It is recommended to set a large seed, i.e. a number that has a good balance of 0 and 1 bits. Avoid having many 0 bits in the seed.

In [0]:


#def train(): #commented it because otherwise the variable would remain local to this
    if torch.cuda.is_available():
        torch.cuda.manual_seed(123)
    else:
        torch.manual_seed(123)

    trainingParams = {"batch_size": 128, # Or larger one
                       "shuffle": True,
                       "num_workers": 0}

    trainingDataset = MyDataset( DIR + "train.csv", 1014)
    trainingLoader = DataLoader(trainingDataset, **trainingParams)

    model = CharacterLevelCNN(input_length=1014, n_classes=trainingDataset.num_classes,
                              input_dim= 70,
                              n_conv_filters=256, n_fc_neurons=1024)
    

    if torch.cuda.is_available():
        model.cuda()

    criterion = nn.CrossEntropyLoss()
  

    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    best_loss = 1e5
    best_epoch = 0
    model.train()
    num_iter_per_epoch = len(trainingLoader)

    for epoch in range(30):
      for iter, batch in enumerate(trainingLoader):
          feature, label = batch
          if torch.cuda.is_available():
              feature = feature.cuda() #loading them on GPU
              label = label.cuda()
          optimizer.zero_grad()
          predictions = model(feature)
          loss = criterion(predictions, label)
          loss.backward()
          optimizer.step()

          training_metrics = get_evaluation(label.cpu().numpy(), predictions.cpu().detach().numpy(),
                                                list_metrics=["accuracy"])                          #read the purpose of .cpu 
          print("Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
              epoch + 1,
              30,
              iter + 1,
              num_iter_per_epoch,
              optimizer.param_groups[0]['lr'],
              loss, training_metrics["accuracy"]))
          # writer.add_scalar('Train/Loss', loss, epoch * num_iter_per_epoch + iter)
          # writer.add_scalar('Train/Accuracy', training_metrics["accuracy"], epoch * num_iter_per_epoch + iter)
     # model.eval()# don't know why????
      if epoch % 3 == 0 and epoch > 0:
            current_lr = optimizer.state_dict()['param_groups'][0]['lr']
            current_lr /= 2
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_lr


Epoch: 1/30, Iteration: 1/938, Lr: 0.01, Loss: 4.807629585266113, Accuracy: 0.2890625
Epoch: 1/30, Iteration: 2/938, Lr: 0.01, Loss: 6.687944412231445, Accuracy: 0.28125
Epoch: 1/30, Iteration: 3/938, Lr: 0.01, Loss: 5.030582904815674, Accuracy: 0.2578125
Epoch: 1/30, Iteration: 4/938, Lr: 0.01, Loss: 2.511899948120117, Accuracy: 0.2265625
Epoch: 1/30, Iteration: 5/938, Lr: 0.01, Loss: 2.9812324047088623, Accuracy: 0.2421875
Epoch: 1/30, Iteration: 6/938, Lr: 0.01, Loss: 1.9108545780181885, Accuracy: 0.2578125
Epoch: 1/30, Iteration: 7/938, Lr: 0.01, Loss: 1.749289631843567, Accuracy: 0.171875
Epoch: 1/30, Iteration: 8/938, Lr: 0.01, Loss: 1.5389268398284912, Accuracy: 0.265625
Epoch: 1/30, Iteration: 9/938, Lr: 0.01, Loss: 1.4584250450134277, Accuracy: 0.265625
Epoch: 1/30, Iteration: 10/938, Lr: 0.01, Loss: 1.455443263053894, Accuracy: 0.3125
Epoch: 1/30, Iteration: 11/938, Lr: 0.01, Loss: 1.5175330638885498, Accuracy: 0.296875
Epoch: 1/30, Iteration: 12/938, Lr: 0.01, Loss: 1.528381

In [0]:
model.eval()
test_params = {"batch_size": 128,
                   "shuffle": False,
                   "num_workers": 0}

test_set = MyDataset(DIR +"test.csv", 1014)
test_generator = DataLoader(test_set, **test_params)



loss_ls = []
te_label_ls = []
te_pred_ls = []
for batch in test_generator:
            te_feature, te_label = batch
            num_sample = len(te_label)
            if torch.cuda.is_available():
                te_feature = te_feature.cuda()
                te_label = te_label.cuda()
            with torch.no_grad():
                te_predictions = model(te_feature)
            te_loss = criterion(te_predictions, te_label)
            loss_ls.append(te_loss * num_sample)
            te_label_ls.extend(te_label.clone().cpu())
            te_pred_ls.append(te_predictions.clone().cpu())

te_loss = sum(loss_ls) / test_set.__len__()
te_pred = torch.cat(te_pred_ls, 0)
te_label = np.array(te_label_ls)
test_metrics = get_evaluation(te_label, te_pred.numpy(), list_metrics=["accuracy", "confusion_matrix"])
      
print("Epoch: {}/{}, Lr: {}, Loss: {}, Accuracy: {},Confusion Matrix: {}".format(
            2,
            2,
            optimizer.param_groups[0]['lr'],
            te_loss, test_metrics["accuracy"],test_metrics["confusion_matrix"])
)

        


Epoch: 2/2, Lr: 1.953125e-05, Loss: 0.6007241606712341, Accuracy: 0.8678947368421053,Confusion Matrix: [[1657   65   93   85]
 [  61 1785   28   26]
 [  86   26 1572  216]
 [  62   34  222 1582]]


In [0]:
def train():
    if torch.cuda.is_available():
        torch.cuda.manual_seed(123)
    else:
        torch.manual_seed(123)
  

   # if not os.path.exists(opt.output):
    #    os.makedirs(opt.output)
    #output_file = open(opt.output + os.sep + "logs.txt", "w")
    #output_file.write("Model's parameters: {}".format(vars(opt)))

    training_params = {"batch_size": 128,
                       "shuffle": True,
                       "num_workers": 0}
    test_params = {"batch_size": 128,
                   "shuffle": False,
                   "num_workers": 0}
    training_set = MyDataset(DIR + "train.csv", 1014)
    test_set = MyDataset(DIR +"test.csv", 1014)
    training_generator = DataLoader(training_set, **training_params)
    test_generator = DataLoader(test_set, **test_params)

    #if opt.feature == "small":
    model = CharacterLevelCNN(input_length = 1014, n_classes=training_set.num_classes,
                                  input_dim= 70,
                                  n_conv_filters=256, n_fc_neurons=1024)

    
    #writer = SummaryWriter(log_path)

    if torch.cuda.is_available():
        model.cuda()

    criterion = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)
    best_loss = 1e5
    best_epoch = 0
    model.train()
    num_iter_per_epoch = len(training_generator)
    num_epochs = 30
    for epoch in range(num_epochs):
        for iter, batch in enumerate(training_generator):
            feature, label = batch
            if torch.cuda.is_available():
                feature = feature.cuda()
                label = label.cuda()
            optimizer.zero_grad()
            predictions = model(feature)
            loss = criterion(predictions, label)
            loss.backward()
            optimizer.step()

            training_metrics = get_evaluation(label.cpu().numpy(), predictions.cpu().detach().numpy(),
                                              list_metrics=["accuracy"])
            print("Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
                epoch + 1,
                num_epochs,
                iter + 1,
                num_iter_per_epoch,
                optimizer.param_groups[0]['lr'],
                loss, training_metrics["accuracy"]))
            #writer.add_scalar('Train/Loss', loss, epoch * num_iter_per_epoch + iter)
            #writer.add_scalar('Train/Accuracy', training_metrics["accuracy"], epoch * num_iter_per_epoch + iter)
        model.eval()
        loss_ls = []
        te_label_ls = []
        te_pred_ls = []
        for batch in test_generator:
            te_feature, te_label = batch
            num_sample = len(te_label)
            if torch.cuda.is_available():
                te_feature = te_feature.cuda()
                te_label = te_label.cuda()
            with torch.no_grad():
                te_predictions = model(te_feature)
            te_loss = criterion(te_predictions, te_label)
            loss_ls.append(te_loss * num_sample)
            te_label_ls.extend(te_label.clone().cpu())
            te_pred_ls.append(te_predictions.clone().cpu())

        te_loss = sum(loss_ls) / test_set.__len__()
        te_pred = torch.cat(te_pred_ls, 0)
        te_label = np.array(te_label_ls)
        test_metrics = get_evaluation(te_label, te_pred.numpy(), list_metrics=["accuracy", "confusion_matrix"])
        #output_file.write(
            #"Epoch: {}/{} \nTest loss: {} Test accuracy: {} \nTest confusion matrix: \n{}\n\n".format(
            #    epoch + 1, opt.num_epochs,
            #    te_loss,
            #    test_metrics["accuracy"],
            #    test_metrics["confusion_matrix"]))
        print("Epoch: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
            epoch + 1,
            num_epochs,
            optimizer.param_groups[0]['lr'],
            te_loss, test_metrics["accuracy"]))
        #writer.add_scalar('Test/Loss', te_loss, epoch)
        #writer.add_scalar('Test/Accuracy', test_metrics["accuracy"], epoch)
        model.train()
        if te_loss + 0 < best_loss:
            best_loss = te_loss
            best_epoch = epoch
            #torch.save(model, "{}/char-cnn_{}_{}".format(opt.output, opt.dataset, opt.feature))
        # Early stopping
        if epoch - best_epoch > 3 > 0:
            print("Stop training at epoch {}. The lowest loss achieved is {} at epoch {}".format(epoch, te_loss, best_epoch))
            break
        if epoch % 3 == 0 and epoch > 0:
            current_lr = optimizer.state_dict()['param_groups'][0]['lr']
            current_lr /= 2
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_lr




In [0]:
train()

Streaming output truncated to the last 5000 lines.
Epoch: 7/30, Iteration: 637/938, Lr: 0.005, Loss: 0.3082091808319092, Accuracy: 0.8984375
Epoch: 7/30, Iteration: 638/938, Lr: 0.005, Loss: 0.35384613275527954, Accuracy: 0.890625
Epoch: 7/30, Iteration: 639/938, Lr: 0.005, Loss: 0.2747859060764313, Accuracy: 0.8984375
Epoch: 7/30, Iteration: 640/938, Lr: 0.005, Loss: 0.20824238657951355, Accuracy: 0.9140625
Epoch: 7/30, Iteration: 641/938, Lr: 0.005, Loss: 0.273945152759552, Accuracy: 0.8984375
Epoch: 7/30, Iteration: 642/938, Lr: 0.005, Loss: 0.2885242998600006, Accuracy: 0.921875
Epoch: 7/30, Iteration: 643/938, Lr: 0.005, Loss: 0.33262884616851807, Accuracy: 0.859375
Epoch: 7/30, Iteration: 644/938, Lr: 0.005, Loss: 0.38574010133743286, Accuracy: 0.8671875
Epoch: 7/30, Iteration: 645/938, Lr: 0.005, Loss: 0.24899375438690186, Accuracy: 0.90625
Epoch: 7/30, Iteration: 646/938, Lr: 0.005, Loss: 0.27371805906295776, Accuracy: 0.890625
Epoch: 7/30, Iteration: 647/938, Lr: 0.005, Loss: 